In [51]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm 
import sr_processing as sr

In [54]:
main_path = '/scratch/ecephys_714527_2024-05-14_13-22-07_nwb_2025-08-03_21-11-22/'
epoch_path = os.path.join(main_path,'epochs.pkl')
df_epoch = pd.read_pickle(epoch_path)

In [55]:
bin_size = 0.01
#stim_ep_list = ['RFMapping_0','RFMapping_1','RFMapping_2','RFMapping_3']
stim_ep_list = [i for i in df_epoch['stim_name'] if 'RFMapping' in i]
for stims_eps in stim_ep_list:
    epochs_path = f'{main_path}{stims_eps}/'
    fr_path = f'{epochs_path}/firing_rate2/'
    trial_path = os.path.join(epochs_path,'trial_info.csv')
    df_trials = pd.read_csv(trial_path,index_col='Unnamed: 0')
    index_map = sr.get_index_maps(df_trials)
    results_dic ={}
    pbar = tqdm(total = len(index_map.keys()),leave=True, position=0)
    for inds in index_map.keys():
        trials2avg = index_map[inds]
        data_set = {d_ind:sr.get_spikerate_data(f'{fr_path}trial_array_{d_ind}.pkl') for d_ind in trials2avg}
        if len(set([d['bin_centers'].shape[0] for d in data_set.values()])) != 1:
            data_set = sr.trim_binsz_ts(data_set)
        temp_results = sr.psth_cal([trial['s_counts'] for trial in data_set.values()])/bin_size
        results_dic[inds] = sr.spike_rate_zscore(temp_results)
        pbar.update(1)
    filename = f'{epochs_path}firing_rates.pkl'
    pickle.dump(results_dic, open(filename, 'wb'))

100%|██████████| 193/193 [00:16<00:00, 52.35it/s]